In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
x=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/train.csv')
x_test=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-1/test.csv')

In [ ]:
x.head()

In [ ]:
x.describe()

In [ ]:
x.isnull().sum()

In [ ]:
x.corr()

In [ ]:
x1=x.iloc[:,3:5]
x_test2=x_test.iloc[:,3:5]

In [ ]:
x_test.isnull().sum()

In [ ]:
x1

In [ ]:
from fancyimpute import IterativeImputer as MICE

In [ ]:
x1_c = list(x1)

In [ ]:
x1_c

In [ ]:
y=MICE()

In [ ]:
x2 = pd.DataFrame(y.fit_transform(x1))

In [ ]:
x_test3=pd.DataFrame(y.transform(x_test2))

In [ ]:
x2

In [ ]:
x2.columns=x1_c
x_test3.columns=x1_c

In [ ]:
x2.isnull().sum()

In [ ]:
x_test3.isnull().sum()

In [ ]:
x['Lat']=x2['Lat']
x_test['Lat']=x_test3['Lat']

In [ ]:
x['Long']=x2['Long']
x_test['Long']=x_test3['Long']

In [ ]:
x.isnull().sum()

In [ ]:
x['Date']=pd.to_datetime(x['Date'])
x_test['Date']=pd.to_datetime(x_test['Date'])

In [ ]:
x['Date'].describe()

In [ ]:
from datetime import date

In [ ]:
mindate=x['Date'].min()

In [ ]:
mindate

In [ ]:
z=x['Date']-mindate

In [ ]:
for i in range(len(z)):
    z[i]=int(str(z[i]).split()[0])

In [ ]:
x['Date']=z

In [ ]:
x

In [ ]:
c=x_test['Date']-mindate

In [ ]:
for i in range(len(c)):
    c[i]=int(str(c[i]).split()[0])

In [ ]:
c

In [ ]:
x_test['Date']=c

In [ ]:
x['Date']=x['Date'].astype(int)

In [ ]:
x_test['Date']=x_test['Date'].astype(int)

In [ ]:
x_test

In [ ]:
y1_val=x.iloc[:,-2]

In [ ]:
y1_val=y1_val.astype(int)

In [ ]:
y2_val=x.iloc[:,-1]

In [ ]:
y2_val=y2_val.astype(int)

In [ ]:
x_val=x.drop(['ConfirmedCases','Fatalities'],axis=1)

In [ ]:
x_val

In [ ]:
z=x.corr()

In [ ]:
import seaborn as sns;

In [ ]:
sns.heatmap(z)

In [ ]:
# for i in range(len(x)):
z=x_val['Province/State'].isna()


In [ ]:
z1=x_test['Province/State'].isna()

In [ ]:
for i in range(len(z1)):
       if z1[i]==True:
            x_test['Province/State'][i]=x_test['Country/Region'][i]

In [ ]:
x_test.isna().sum()

In [ ]:
for i in range(len(z)):
       if z[i]==True:
            x_val['Province/State'][i]=x_val['Country/Region'][i]

In [ ]:
x_val.isnull().sum()

In [ ]:
x.drop(['ConfirmedCases','Fatalities'],axis=1,inplace=True)

In [ ]:
x

In [ ]:
x_val.isnull().sum()

In [ ]:
xx=pd.get_dummies(x_val)

In [ ]:
x_predfinal=pd.get_dummies(x_test)

In [ ]:
x_predfinal

In [ ]:
xx

In [ ]:
sns.heatmap(xx.corr())

In [ ]:
# xx.to_csv('hihi.csv')

In [ ]:
from sklearn.ensemble import BaggingRegressor

In [ ]:
from sklearn import tree

In [ ]:

from sklearn.model_selection import train_test_split


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(xx, y1_val, test_size=0.2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(xx, y2_val, test_size=0.2)

In [ ]:
X_test1

In [ ]:
model = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1))

In [ ]:
model.fit(X_train1, y_train1)

In [ ]:
model.score(X_test1,y_test1)

In [ ]:
# mio=pd.read_csv('')

In [ ]:
anssss=model.predict(x_predfinal)

In [ ]:
anssss

In [ ]:
a1=np.around(anssss)

In [ ]:
a1=a1.astype(int)

In [ ]:
a1

In [ ]:
model2 = BaggingRegressor(tree.DecisionTreeRegressor(random_state=1))

In [ ]:
model2.fit(X_train2, y_train2)

In [ ]:
model2.score(X_test2,y_test2)

In [ ]:
anzzzz=model2.predict(x_predfinal)

In [ ]:
a2=np.around(anzzzz)

In [ ]:
a2=a2.astype(int)

In [ ]:
predictions = pd.DataFrame({'ForecastId':x_test['ForecastId'],'ConfirmedCases':a1,'Fatalities':a2})
predictions.head()

In [ ]:
# predictions.to_csv('submission.csv', header=True, index=False)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
model3 = AdaBoostRegressor()
model4 = AdaBoostRegressor()

In [ ]:
model3.fit(X_train1, y_train1)

In [ ]:
model4.fit(X_train2, y_train2)

In [ ]:
model3.score(X_test1,y_test1)

In [ ]:
model4.score(X_test2,y_test2)

In [ ]:
import xgboost as xgb

In [ ]:
model5=xgb.XGBRegressor()

In [ ]:
model6=xgb.XGBRegressor()

In [ ]:
X_train1.rename(columns = {'Id':'ForecastId'}, inplace = True) 
X_test1.rename(columns = {'Id':'ForecastId'}, inplace = True) 

In [ ]:
X_train2.rename(columns = {'Id':'ForecastId'}, inplace = True) 
X_test2.rename(columns = {'Id':'ForecastId'}, inplace = True) 

In [ ]:
model5.fit(X_train1, y_train1)

In [ ]:
model6.fit(X_train2, y_train2)

In [ ]:
model5.score(X_test1,y_test1)

In [ ]:
model6.score(X_test2,y_test2)

In [ ]:
ank=model5.predict(x_predfinal)
annk=model6.predict(x_predfinal)

a4=np.around(ank)
a4=a4.astype(int)

a5=np.around(annk)
a5=a5.astype(int)

predictions2 = pd.DataFrame({'ForecastId':x_test['ForecastId'],'ConfirmedCases':a4,'Fatalities':a5})
predictions2.to_csv('submission.csv', header=True, index=False)
predictions2.head()

In [ ]:
x_test

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_predmodel5=model5.predict(X_test1)

In [ ]:
y_predmodel5

In [ ]:
mean_squared_error(y_test1, y_predmodel5)